In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Algorithme de decision tree

1. Choix de la variable racine
celle qui permet de mieux séparer les données
- teste chaque variable seule
- sélectionne celle qui donne des feuilles avec la plus faible impureté

**Feuille pure** ne contient que des 1 ou que des 0
**Feuille impure** contient un mélange de 1 et de 0

**Gini Impurity d’une feuille** = $
1 - P(\text{oui})^2 - P(\text{non})^2
$

In [ ]:
df = pd.read_csv('../data/processed/train_optimized.csv')
y = df['target']
X = df.drop(["target", "id"], axis=1)

In [3]:
X.head(1)

,keyword,text_cleaned,text_length,word_count,char_count,has_emergency_word,emergency_word_count,emergency_density,has_url,url_count,has_mention,mention_count,exclamation_count,intense_punctuation,avg_word_length,urgency_score,stopword_ratio,keyword_in_text
0,forest%20fires,a little concerned about the number of forest ...,72,14,72,False,0,0.0,False,0,False,0,0,0,4.214286,0.0,0.428571,False


In [4]:
df.head()

,id,keyword,target,text_cleaned,text_length,word_count,char_count,has_emergency_word,emergency_word_count,emergency_density,has_url,url_count,has_mention,mention_count,exclamation_count,intense_punctuation,avg_word_length,urgency_score,stopword_ratio,keyword_in_text
0,5744,forest%20fires,1,a little concerned about the number of forest ...,72,14,72,False,0,0.000000,False,0,False,0,0,0,4.214286,0.0,0.428571,False
1,4178,drown,0,when a real nigga hold you down you supposed t...,53,11,53,False,0,0.000000,False,0,False,0,0,0,3.909091,0.0,0.454545,True
2,109,accident,0,rt mention_token sleeping pills double your ri...,90,12,76,True,1,0.083333,True,1,True,1,0,0,5.416667,1.0,0.250000,True
3,5076,famine,1,new article russian food crematoria provoke ou...,123,16,107,True,1,0.062500,True,1,False,0,0,0,5.750000,0.5,0.125000,True
4,5942,hazard,0,seeing hazard without the beard like... url_token,62,7,49,False,0,0.000000,True,1,False,0,0,0,6.142857,1.0,0.142857,True


### Lister les gini impurity de chaque variable 
PAS UTILE de créer un fichier<br>
un dictionnaire en mémoire suffit


In [ ]:
gini_scores = {}

def gini(group):
    """
    Calcule l'impureté de Gini pour un groupe de valeurs cibles binaires (0/1).
    Plus la valeur est proche de 0, plus le groupe est pur.
    """
    n = len(group)
    if n == 0:
        return 0
    p_counts = group.value_counts(normalize=True)
    return 1 - sum(p_counts**2)

### Variables booléennes

In [ ]:
def bool_lower_impurity(df, col, target='target'):
    """
    Calcule l'impureté de Gini pour un split sur une variable booléenne.
    Retourne :
        - l'impureté du groupe True
        - l'impureté du groupe False
        - l'impureté pondérée globale du split
    """
    # Sépare les True et les False et récupère les valeurs de la cible
    true_group = df[df[col]][target]
    false_group = df[~df[col]][target]

    total = len(df)

    true_group_gini = gini(true_group)
    false_group_gini = gini(false_group)

    # Gini impurity total pondéré (utile pour le split global)
    weighted_gini = (
        len(true_group) / total * true_group_gini +
        len(false_group) / total * false_group_gini
)

    return true_group_gini, false_group_gini, weighted_gini


In [ ]:
for col in df.select_dtypes(include='bool'):
        gini_true, gini_false, gini_weighted = bool_lower_impurity(df, col, target='target')
        print(f"{col}\ngini(True) = {gini_true}, gini(False) = {gini_false}, gini(weighted) = {gini_weighted}\n")
        gini_scores[col] = {
        'gini_true': gini_true,
        'gini_false': gini_false,
        'gini_weighted': gini_weighted
    }

has_emergency_word
gini(True) = 0.48105339999817875, gini(False) = 0.42421029533532995, gini(weighted) = 0.4441812034084699

has_url
gini(True) = 0.49859080660310306, gini(False) = 0.41721759809750303, gini(weighted) = 0.45662143956464485

has_mention
gini(True) = 0.44089292603278607, gini(False) = 0.49245439644109057, gini(weighted) = 0.4781489019834268

keyword_in_text
gini(True) = 0.47552666614456895, gini(False) = 0.49808738115854334, gini(weighted) = 0.4807756344396553



### Variables numérique

In [13]:
def numeric_lower_impurity(df, col, target='target', n_thresholds=10):
    """
    Cherche le meilleur seuil pour splitter une variable numérique afin de minimiser l'impureté de Gini.
    Teste plusieurs seuils (valeurs uniques ou quantiles).
    Retourne :
        - le meilleur seuil
        - la Gini impurity pondérée associée
    """
    unique_vals = sorted(df[col].dropna().unique())
    if len(unique_vals) <= 1:
        return None  # Pas de split possible

    # Choix des seuils : valeurs uniques ou quantiles si trop de valeurs
    thresholds = unique_vals
    if len(unique_vals) > n_thresholds:
        thresholds = np.linspace(min(unique_vals), max(unique_vals), n_thresholds + 2)[1:-1]

    best_gini = float('inf')
    best_threshold = None

    for thresh in thresholds:
        left = df[df[col] <= thresh][target]
        right = df[df[col] > thresh][target]
        total = len(df)
        weighted_gini = (
            len(left) / total * gini(left) +
            len(right) / total * gini(right)
        )
        if weighted_gini < best_gini:
            best_gini = weighted_gini
            best_threshold = thresh

    return best_threshold, best_gini


### Variables texte
Il faut les convertir en variables numériques : BagOfWords, TF_IDF, Word2Vec, ...

In [14]:
class DecisionTreeNode:
    """
    Noeud d'un arbre de décision binaire.

    Attributs :
        - feature : nom de la variable utilisée pour le split (None pour une feuille)
        - threshold : seuil utilisé pour le split numérique (None pour booléen ou feuille)
        - left : sous-arbre gauche (valeurs <= threshold ou True)
        - right : sous-arbre droit (valeurs > threshold ou False)
        - value : valeur prédite si feuille (None sinon)
    """
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        """Retourne True si le noeud est une feuille (pas de split)."""
        return self.value is not None

In [15]:
class DecisionTreeClassifierSimple:
    """
    Implémentation simple d'un classifieur arbre de décision binaire (pour variables booléennes et numériques).
    - max_depth : profondeur maximale de l'arbre
    - min_samples_split : nombre minimal d'échantillons pour effectuer un split
    """
    def __init__(self, max_depth=3, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, df, target='target', depth=0):
        """
        Apprend récursivement l'arbre sur le DataFrame df.
        Sélectionne à chaque étape la variable et le split qui minimisent l'impureté de Gini.
        Arrête si :
            - toutes les cibles sont identiques
            - profondeur max atteinte
            - trop peu d'échantillons
        Retourne la racine du sous-arbre construit.
        """
        y = df[target]
        if len(set(y)) == 1 or depth >= self.max_depth or len(df) < self.min_samples_split:
            return DecisionTreeNode(value=y.mode()[0])

        best_gini = float('inf')
        best_col = None
        best_threshold = None
        best_sides = None

        for col in df.columns:
            if col == target or col == 'id':
                continue
            if df[col].dtype == bool:
                left = df[df[col] == True]
                right = df[df[col] == False]
                gini_left = gini(left[target])
                gini_right = gini(right[target])
                weighted_gini = (len(left) * gini_left + len(right) * gini_right) / len(df)
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_col = col
                    best_threshold = None
                    best_sides = (left, right)
            elif np.issubdtype(df[col].dtype, np.number):
                result = numeric_lower_impurity(df, col, target=target)
                if result:
                    threshold, gini_value = result
                    left = df[df[col] <= threshold]
                    right = df[df[col] > threshold]
                    if gini_value < best_gini:
                        best_gini = gini_value
                        best_col = col
                        best_threshold = threshold
                        best_sides = (left, right)
        if best_col is None:
            return DecisionTreeNode(value=y.mode()[0])
        left_node = self.fit(best_sides[0], target, depth+1)
        right_node = self.fit(best_sides[1], target, depth+1)
        return DecisionTreeNode(feature=best_col, threshold=best_threshold, left=left_node, right=right_node)

    def train(self, df, target='target'):
        """Entraîne l'arbre sur le DataFrame df (wrapper pour fit)."""
        self.root = self.fit(df, target)

    def predict_one(self, x, node=None):
        """
        Prédit la classe pour une observation x (série pandas).
        Parcourt l'arbre récursivement jusqu'à une feuille.
        """
        if node is None:
            node = self.root
        if node.is_leaf():
            return node.value
        if node.threshold is None:
            return self.predict_one(x, node.left) if x[node.feature] else self.predict_one(x, node.right)
        else:
            return self.predict_one(x, node.left) if x[node.feature] <= node.threshold else self.predict_one(x, node.right)

    def predict(self, X):
        """Prédit la classe pour chaque ligne du DataFrame X."""
        return X.apply(lambda row: self.predict_one(row, self.root), axis=1)

In [16]:
# Entraînement et prédiction avec l'arbre de décision simple
clf = DecisionTreeClassifierSimple(max_depth=3, min_samples_split=2)
clf.train(df, target='target')
preds = clf.predict(df.drop(["target", "id"], axis=1))
print("Prédictions sur l'ensemble d'entraînement :")
print(preds.value_counts())

Prédictions sur l'ensemble d'entraînement :
0    4801
1    1384
Name: count, dtype: int64
